In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import numpy as np
import requests as rq
import io, h5py

In [2]:
data = rq.get('https://www.dropbox.com/s/c3umbo5y13sqcfp/synthetic_dataset.h5?raw=true')
data.raise_for_status()

with h5py.File(io.BytesIO(data.content), 'r') as dataset:
    x_train = np.array(dataset['X_train']).astype(np.float32).transpose([0, 2, 1])
    y_train = np.array(dataset['Y_train']).astype(np.float32)
    x_valid = np.array(dataset['X_valid']).astype(np.float32).transpose([0, 2, 1])
    y_valid = np.array(dataset['Y_valid']).astype(np.int32)
    x_test = np.array(dataset['X_test']).astype(np.float32).transpose([0, 2, 1])
    y_test = np.array(dataset['Y_test']).astype(np.int32)

print(x_train.shape, y_train.shape)

(21000, 200, 4) (21000, 12)


In [3]:
def generate_model():
  input_layer = layers.Input(shape=(200, 4))

  conv1 = layers.Conv1D(filters=64, kernel_size=19, padding='same', use_bias='false')(input_layer)
  BN1 = layers.BatchNormalization()(conv1)
  relu1 = layers.Activation('relu')(BN1)
  pool1 = layers.MaxPool1D(pool_size=25)(relu1)
  drop1 = layers.Dropout(0.05)(pool1)

  positions = tf.range(drop1.shape[1])
  context = layers.Embedding(input_dim=drop1.shape[1], output_dim=drop1.shape[2])(positions)

  contextual_meaning = tf.add(drop1, context)

  query = layers.Dense(100, name='query')(contextual_meaning)
  value = layers.Dense(100)(contextual_meaning)

  attention, weights = layers.MultiHeadAttention(num_heads=8, key_dim=32, dropout=0.55)(query, value, return_attention_scores=True)

  flat = layers.Flatten()(attention)
  fcl = layers.Dense(512, use_bias=False)(flat)
  BN2 = layers.BatchNormalization()(fcl)
  relu2 = layers.Activation('relu')(BN2)
  drop2 = layers.Dropout(0.5)(relu2)

  output = layers.Dense(12, activation='sigmoid')(drop2)

  model = Model(inputs=input_layer, outputs=output)
  model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(curve='ROC', name='auroc'), tf.keras.metrics.AUC(curve='PR', name='aupr')])
  return model

In [10]:
model = generate_model()
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='min', restore_best_weights=False)
lr_decay = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-10, mode='min', verbose=1) 
model.fit(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid), callbacks=[early_stop, lr_decay])

Epoch 1/100
657/657 [==============================] - 7s 8ms/step - loss: 0.5556 - auroc: 0.5078 - aupr: 0.1489 - val_loss: 0.4527 - val_auroc: 0.5493 - val_aupr: 0.1713
Epoch 2/100
657/657 [==============================] - 5s 7ms/step - loss: 0.4666 - auroc: 0.5297 - aupr: 0.1597 - val_loss: 0.4287 - val_auroc: 0.5943 - val_aupr: 0.2448
Epoch 3/100
657/657 [==============================] - 5s 7ms/step - loss: 0.4365 - auroc: 0.5904 - aupr: 0.2243 - val_loss: 0.3944 - val_auroc: 0.6886 - val_aupr: 0.3497
Epoch 4/100
657/657 [==============================] - 5s 7ms/step - loss: 0.4068 - auroc: 0.6509 - aupr: 0.3089 - val_loss: 0.3419 - val_auroc: 0.7681 - val_aupr: 0.4824
Epoch 5/100
657/657 [==============================] - 5s 7ms/step - loss: 0.3682 - auroc: 0.7216 - aupr: 0.4147 - val_loss: 0.3001 - val_auroc: 0.8306 - val_aupr: 0.5809
Epoch 6/100
657/657 [==============================] - 5s 7ms/step - loss: 0.3349 - auroc: 0.7764 - aupr: 0.5011 - val_loss: 0.2826 - val_auroc: 